In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [3]:
import warnings
warnings.filterwarnings('ignore')

#  Смотрим на данные в целом

In [4]:
train_df = pd.read_csv('/Users/antosale/Downloads/realestatepriceprediction/train.csv').set_index('Id')
train_df.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
Id,,,,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [5]:
train_df.dtypes

DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [6]:
train_df.shape

(10000, 19)

In [7]:
train_df.isnull().sum()

DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [8]:
train_df.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


# Анализируем категориальные признаки

In [9]:
# Преобразуем номер района в тип str, так как он сам ни на что не влияет
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

In [10]:
# Проверим категориальные фичи на уникальные значения
print(train_df['Ecology_2'].unique())
print(train_df['Ecology_3'].unique())
print(train_df['Shops_2'].unique())

['B' 'A']
['B' 'A']
['B' 'A']


In [11]:
# Замением их на 0 и 1
train_df['Ecology_2'] = train_df['Ecology_2'].replace(['A', 'B'],[0, 1])
train_df['Ecology_3'] = train_df['Ecology_3'].replace(['A', 'B'],[0, 1])
train_df['Shops_2'] = train_df['Shops_2'].replace(['A', 'B'],[0, 1])

In [12]:
train_df.dtypes

DistrictId        object
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2          int64
Ecology_3          int64
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2            int64
Price            float64
dtype: object

# Начинаем обрабатывать выбросы

In [13]:
steps = []

In [14]:
# Отбрасываем лишние значения
steps.append('обработка пропусков, выбросов var1')

train_df = train_df[train_df['Square'].isnull() |
                    (train_df['Square'] < train_df['Square'].quantile(.99)) &
                    (train_df['Square'] > train_df['Square'].quantile(.01))]

train_df = train_df[train_df['LifeSquare'].isnull() |
                    (train_df['LifeSquare'] < train_df['LifeSquare'].quantile(.99)) &
                    (train_df['LifeSquare'] > train_df['LifeSquare'].quantile(.01))]

train_df = train_df[train_df['KitchenSquare'].isnull() |
                    (train_df['KitchenSquare'] < train_df['KitchenSquare'].quantile(.99)) &
                    (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.01))]

In [15]:
train_df.describe(include='all')

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
count,8881,8881.000000,8881.000000,7296.000000,8881.000000,8881.000000,8881.000000,8.881000e+03,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000,4799.000000,8881.000000,8881.000000,8881.00000,8881.000000
unique,205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.888076,54.809487,34.830304,6.075329,8.446684,12.826934,4.242532e+03,0.119620,0.989979,0.969936,26.071163,5630.480014,8.403783,1156.072932,1.392636,4.438577,0.92940,214939.479049
std,NaN,0.826691,16.225349,14.280535,3.503611,5.133763,6.369206,2.127571e+05,0.119654,0.099610,0.170774,17.269698,3977.902810,24.295214,1032.814421,1.492092,4.837340,0.25617,89654.209673
min,NaN,0.000000,29.719934,2.941437,1.000000,1.000000,0.000000,1.914000e+03,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,0.00000,59174.778028
25%,NaN,1.000000,41.637555,22.863805,1.000000,4.000000,9.000000,1.972000e+03,0.017647,1.000000,1.000000,10.000000,2370.000000,0.000000,325.000000,0.000000,1.000000,1.00000,156934.440088
50%,NaN,2.000000,51.656012,32.459931,6.000000,7.000000,14.000000,1.977000e+03,0.075779,1.000000,1.000000,25.000000,5562.000000,2.000000,990.000000,1.000000,3.000000,1.00000,194240.534503
75%,NaN,2.000000,64.720916,44.119769,9.000000,12.000000,17.000000,2.000000e+03,0.194489,1.000000,1.000000,36.000000,7614.000000,5.000000,1548.000000,3.000000,6.000000,1.00000,250381.580780


In [16]:
for columns in list(train_df)[1:]:
    print(train_df[columns].sort_values(ascending=False).value_counts())
    
#Выбросы детектед:
#Rooms: 0, 10, 6, 19
#LifeSquare < 1
#KitchenSquare = 1 (где 0, возможно, без кухни или совмещена с гостинной)
#Ecology_2, Ecology_3, Shops_2 очень ненормированные значения, можно выкинуть столбцы

2.0     3533
1.0     3250
3.0     1973
4.0      112
5.0        8
10.0       2
0.0        1
6.0        1
19.0       1
Name: Rooms, dtype: int64
81.053044    1
44.078793    1
55.100810    1
46.363496    1
72.295749    1
            ..
65.470153    1
65.938754    1
44.053821    1
46.791690    1
97.664912    1
Name: Square, Length: 8881, dtype: int64
35.812832    1
19.363106    1
22.046646    1
50.277491    1
49.942477    1
            ..
22.134412    1
31.349324    1
28.701777    1
21.717610    1
33.743934    1
Name: LifeSquare, Length: 7296, dtype: int64
1.0     2284
8.0     1301
5.0     1153
10.0    1050
6.0     1028
9.0      841
7.0      606
12.0     241
11.0     227
13.0      64
14.0      46
4.0       35
3.0        4
2.0        1
Name: KitchenSquare, dtype: int64
4     838
5     794
3     781
6     727
7     681
8     578
9     532
2     520
10    495
11    411
12    398
13    361
14    315
15    254
16    248
17    224
1     203
18    164
19    104
20     54
21     50
22     44
24   

In [17]:
train_df = train_df.drop(columns=['Ecology_2', 'Ecology_3', 'Shops_2'])

In [18]:
train_df['HouseYear'].sort_values().unique()

array([    1914,     1916,     1917,     1918,     1919,     1928,
           1929,     1930,     1931,     1932,     1933,     1934,
           1935,     1936,     1937,     1938,     1939,     1940,
           1941,     1942,     1947,     1948,     1950,     1951,
           1952,     1953,     1954,     1955,     1956,     1957,
           1958,     1959,     1960,     1961,     1962,     1963,
           1964,     1965,     1966,     1967,     1968,     1969,
           1970,     1971,     1972,     1973,     1974,     1975,
           1976,     1977,     1978,     1979,     1980,     1981,
           1982,     1983,     1984,     1985,     1986,     1987,
           1988,     1989,     1990,     1991,     1992,     1993,
           1994,     1995,     1996,     1997,     1998,     1999,
           2000,     2001,     2002,     2003,     2004,     2005,
           2006,     2007,     2008,     2009,     2010,     2011,
           2012,     2013,     2014,     2015,     2016,     2

In [19]:
train_df.loc[train_df['LifeSquare'] > train_df['Square']].shape
#Возможно, в этих случаях в жилой плащади включены балконы или туалеты), поэтому решил их не исправлять

(346, 16)

In [20]:
train_df.loc[train_df['LifeSquare'] > 7000]
#А вот это явно нужно поправить

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
Id,,,,,,,,,,,,,,,,


In [21]:
train_df.loc[train_df['HouseYear'] == 20052011, 'HouseYear'] = 2011
train_df.loc[train_df['HouseYear'] > 2020, 'HouseYear'] = 2020

In [22]:
train_df.isnull().sum()

DistrictId          0
Rooms               0
Square              0
LifeSquare       1585
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4082
Helthcare_2         0
Shops_1             0
Price               0
dtype: int64

In [23]:
# Заменяю nan на медиану, так как Imputer не работал

f_columns = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor',
            'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Helthcare_2','Helthcare_1', 'Shops_1',]

for column in list(train_df):
    if column in f_columns:
        train_df.loc[train_df[column].isnull(), column] = train_df[column].median()

In [24]:
# train_df.loc[train_df['LifeSquare'] < 1] = np.nan
# train_df.loc[train_df['LifeSquare'] == 1] = np.nan
# train_df.loc[train_df['KitchenSquare'].isin([1970, 2014]), 'KitchenSquare'] = np.nan
# train_df.loc[train_df['LifeSquare'] > 7000] = np.nan

# Feature Engineering

In [25]:
# Во-первых, добавлю "высотность" района через нахождение медианы по этажности домов
district_height = pd.pivot_table(train_df, index=['DistrictId'], aggfunc='median').rename(columns={'HouseFloor':'MedHeight'}).reset_index()
district_merge = district_height[['DistrictId', 'MedHeight']]
district_merge.head()

,DistrictId,MedHeight
0,0,12.0
1,1,17.0
2,10,12.0
3,100,8.0
4,101,5.0


In [26]:
train_df = train_df.merge(district_merge, on='DistrictId', how='left')
train_df.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,MedHeight
0,35,2.0,47.981561,29.442751,6.0,7.0,9.0,1969.0,0.089040,33.0,7976.0,5,NaN,0.0,11.0,184966.930730,12.0
1,41,3.0,65.683640,40.049543,8.0,7.0,9.0,1978.0,0.000070,46.0,10309.0,1,240.0,1.0,16.0,300009.450063,12.0
2,58,2.0,53.352981,52.731512,9.0,8.0,17.0,1977.0,0.437885,23.0,5735.0,3,1084.0,0.0,5.0,175616.227217,12.0
3,99,1.0,39.649192,23.776169,7.0,11.0,12.0,1976.0,0.012339,35.0,5776.0,1,2078.0,2.0,4.0,150226.531644,9.0
4,59,3.0,80.384479,46.683720,12.0,5.0,17.0,2011.0,0.309479,35.0,7715.0,4,990.0,0.0,6.0,215898.447742,17.0


In [27]:
# И во-вторых найду медиану по всем соц и эко признаком по каждому району
kpi_merge = pd.pivot_table(train_df, index='DistrictId', aggfunc='median').reset_index()\
.rename(columns={'Social_1':'MedSocial_1', 'Social_2':'MedSocial_2', 'Social_3':'MedSocial_3', 'Shops_1':'MedShops_1'})

kpi_merge = kpi_merge[['DistrictId', 'MedSocial_1', 'MedSocial_2', 'MedSocial_3', 'MedShops_1']]

In [28]:
train_df = train_df.merge(kpi_merge, on='DistrictId')
train_df.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,MedHeight,MedSocial_1,MedSocial_2,MedSocial_3,MedShops_1
0,35,2.0,47.981561,29.442751,6.0,7.0,9.0,1969.0,0.08904,33.0,...,5,NaN,0.0,11.0,184966.930730,12.0,33.0,7976.0,5.0,11.0
1,35,3.0,63.851403,47.245735,6.0,7.0,9.0,1971.0,0.08904,33.0,...,5,NaN,0.0,11.0,242403.715876,12.0,33.0,7976.0,5.0,11.0
2,35,2.0,43.325817,22.487877,6.0,7.0,14.0,1969.0,0.08904,33.0,...,5,NaN,0.0,11.0,194175.395111,12.0,33.0,7976.0,5.0,11.0
3,35,1.0,46.287195,32.459931,1.0,13.0,12.0,2015.0,0.08904,33.0,...,5,NaN,0.0,11.0,195895.983219,12.0,33.0,7976.0,5.0,11.0
4,35,2.0,48.249779,29.473633,6.0,3.0,9.0,1971.0,0.08904,33.0,...,5,NaN,0.0,11.0,181020.120527,12.0,33.0,7976.0,5.0,11.0


# Строим модель

In [29]:
train_df.isnull().sum()

DistrictId          0
Rooms               0
Square              0
LifeSquare          0
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4082
Helthcare_2         0
Shops_1             0
Price               0
MedHeight           0
MedSocial_1         0
MedSocial_2         0
MedSocial_3         0
MedShops_1          0
dtype: int64

In [30]:
train_df.columns.tolist()

['DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Social_1',
 'Social_2',
 'Social_3',
 'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 'Price',
 'MedHeight',
 'MedSocial_1',
 'MedSocial_2',
 'MedSocial_3',
 'MedShops_1']

In [31]:
feature_names = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor',
                 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
                 'Shops_1','MedHeight','MedSocial_1','MedSocial_2', 'MedSocial_3', 'MedShops_1']

target_name = 'Price'

In [32]:
X = train_df[feature_names]
y = train_df[target_name]

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=21)

In [34]:
X_train.head()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,MedHeight,MedSocial_1,MedSocial_2,MedSocial_3,MedShops_1
6210,2.0,58.171871,33.842101,11.0,4.0,7.0,1998.0,0.007720,38.0,4788.0,2,0.0,4.0,9.0,38.0,4788.0,2.0,4.0
3200,2.0,45.625973,33.689888,5.0,6.0,5.0,1964.0,0.000000,36.0,6714.0,2,0.0,2.0,9.0,36.0,6714.0,2.0,2.0
2376,3.0,79.227116,48.010346,10.0,17.0,16.0,1981.0,0.037178,52.0,11217.0,1,1.0,7.0,9.0,52.0,11217.0,1.0,7.0
8745,1.0,86.988870,86.808704,1.0,8.0,1.0,1977.0,0.060753,15.0,2787.0,2,0.0,7.0,17.0,15.0,2787.0,2.0,7.0
172,1.0,40.655911,21.726820,9.0,5.0,16.0,1975.0,0.050756,24.0,5469.0,1,0.0,1.0,12.0,23.0,5602.0,3.0,5.0


In [35]:
X_train.shape

(6216, 18)

In [36]:
y.shape

(8881,)

In [37]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6216 entries, 6210 to 5327
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          6216 non-null   float64
 1   Square         6216 non-null   float64
 2   LifeSquare     6216 non-null   float64
 3   KitchenSquare  6216 non-null   float64
 4   Floor          6216 non-null   float64
 5   HouseFloor     6216 non-null   float64
 6   HouseYear      6216 non-null   float64
 7   Ecology_1      6216 non-null   float64
 8   Social_1       6216 non-null   float64
 9   Social_2       6216 non-null   float64
 10  Social_3       6216 non-null   int64  
 11  Helthcare_2    6216 non-null   float64
 12  Shops_1        6216 non-null   float64
 13  MedHeight      6216 non-null   float64
 14  MedSocial_1    6216 non-null   float64
 15  MedSocial_2    6216 non-null   float64
 16  MedSocial_3    6216 non-null   float64
 17  MedShops_1     6216 non-null   float64
dtypes: fl

In [38]:
rf_model = RandomForestRegressor(random_state=21)
rf_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=21, verbose=0, warm_start=False)

In [39]:
y_train_preds = rf_model.predict(X_train)
y_test_preds = rf_model.predict(X_val)

print(f'R2 Score для тренировочного датасета: {r2(y_train, y_train_preds):.2f}')
print(f'R2 Score для валидационного датасета: {r2(y_val, y_test_preds):.2f}')

R2 Score для тренировочного датасета: 0.96
R2 Score для валидационного датасета: 0.73


# Кросс валидация

In [40]:
cv_score = cross_val_score(rf_model, X, y, scoring='r2', cv=KFold(n_splits=5, shuffle=True, random_state=21))
cv_score

array([0.72369574, 0.73869588, 0.71783491, 0.68985422, 0.73000169])

In [41]:
cv_score.mean()

0.7200164879644145

# Прогоняем все то же самое на тестовом датасете

In [42]:
test_df = pd.read_csv('/Users/antosale/Downloads/realestatepriceprediction/test.csv')
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [43]:
test_df['DistrictId'] = test_df['DistrictId'].astype(str)
test_df = test_df.drop(columns=['Ecology_2', 'Ecology_3', 'Shops_2'])
test_df.loc[test_df['HouseYear'] == 20052011, 'HouseYear'] = 2011
test_df.loc[test_df['HouseYear'] > 2020, 'HouseYear'] = 2020

In [44]:
f_columns = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor',
            'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Helthcare_2','Helthcare_1', 'Shops_1',]

for column in list(test_df):
    if column in f_columns:
        test_df.loc[test_df[column].isnull(), column] = test_df[column].median()

In [45]:
district_height = pd.pivot_table(test_df, index=['DistrictId'], aggfunc='median').rename(columns={'HouseFloor':'MedHeight'}).reset_index()
district_merge = district_height[['DistrictId', 'MedHeight']]
district_merge.head()

,DistrictId,MedHeight
0,0,9.0
1,1,17.0
2,10,12.0
3,100,9.0
4,101,9.0


In [46]:
test_df = test_df.merge(district_merge, on='DistrictId', how='left')
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,MedHeight
0,725,58,2.0,49.882643,33.432782,6.0,6.0,14.0,1972.0,0.310199,11.0,2748.0,1,NaN,0.0,0.0,12.0
1,15856,74,2.0,69.263183,32.925087,1.0,6.0,1.0,1977.0,0.075779,6.0,1437.0,3,NaN,0.0,2.0,17.0
2,5480,190,1.0,13.597819,15.948246,12.0,2.0,5.0,1909.0,0.000000,30.0,7538.0,87,4702.0,5.0,5.0,7.0
3,15664,47,2.0,73.046609,51.940842,9.0,22.0,22.0,2007.0,0.101872,23.0,4583.0,3,NaN,3.0,3.0,14.0
4,14275,27,1.0,47.527111,43.387569,1.0,17.0,17.0,2017.0,0.072158,2.0,629.0,1,NaN,0.0,0.0,14.0


In [47]:
kpi_merge = pd.pivot_table(test_df, index='DistrictId', aggfunc='median').reset_index()\
.rename(columns={'Social_1':'MedSocial_1', 'Social_2':'MedSocial_2', 'Social_3':'MedSocial_3', 'Shops_1':'MedShops_1'})

kpi_merge = kpi_merge[['DistrictId', 'MedSocial_1', 'MedSocial_2', 'MedSocial_3', 'MedShops_1']]

In [48]:
test_df = test_df.merge(kpi_merge, on='DistrictId')
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,MedHeight,MedSocial_1,MedSocial_2,MedSocial_3,MedShops_1
0,725,58,2.0,49.882643,33.432782,6.0,6.0,14.0,1972.0,0.310199,...,2748.0,1,NaN,0.0,0.0,12.0,23.0,5735.0,3.0,5.0
1,11160,58,1.0,48.610661,48.752502,1.0,4.0,3.0,1977.0,0.437885,...,5735.0,3,1084.0,0.0,5.0,12.0,23.0,5735.0,3.0,5.0
2,2292,58,1.0,42.363034,23.001954,9.0,8.0,14.0,1976.0,0.050756,...,5469.0,1,145.0,0.0,1.0,12.0,23.0,5735.0,3.0,5.0
3,7953,58,3.0,73.252333,49.068642,9.0,5.0,9.0,2010.0,0.310199,...,2748.0,1,NaN,0.0,0.0,12.0,23.0,5735.0,3.0,5.0
4,2860,58,2.0,62.764993,33.654133,11.0,7.0,10.0,2003.0,0.310199,...,2748.0,1,NaN,0.0,0.0,12.0,23.0,5735.0,3.0,5.0


In [49]:
y_test_preds = rf_model.predict(test_df[feature_names])
test_df['Price'] = y_test_preds



In [50]:
sub = test_df[['Id', 'Price']]
sub.to_csv('NVAntoshin_predictions.csv', index=False)

In [51]:
s = pd.read_csv('NVAntoshin_predictions.csv')
s

,Id,Price
0,725,162637.648111
1,11160,116668.315928
2,2292,175576.378987
3,7953,276588.191838
4,2860,221180.508221
...,...,...
4995,1143,286468.229122
4996,6445,266480.089088
4997,14024,177234.288196
4998,12234,193317.721831
